## 토픽모델링 : 문서집합의 주제 찾기
* 사람은 문장을 요약하는 것에 비해, ML은 주제를 효과적 표현하는 중심단어 추출
* LSA(Latent Semantic Analysis), LDA(Latent Dirichlet Allocation)
 cf) 차원축소의 LDA(Linear Discriminant Analysis) 다름
* 20뉴스그룹 데이터 사용 -> count기반 벡터화(fit_transform) -> lda.fit =>
  - components_속성 (개별 토픽별 각 word 피처가 얼마나 많이 그 토픽에 할당됐는지에 대한 수치)

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출. 
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

# 위에서 cats 변수로 기재된 category만 추출. featch_20newsgroups( )의 categories에 cats 입력
news_df= fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), 
                            categories=cats, random_state=0)

#LDA 는 Count기반의 Vectorizer만 적용합니다.  
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


In [20]:
print(f'{type(news_df)}')
# news_df => 데이터의 형태가 dictionary
# print(news_df.shape)  => error
print(f'{news_df.keys()}\n')

print(len(news_df.data)) # 7862   #news_df.data => list [ , , , ]
print(news_df.data[0])

print(news_df.filenames) # 8

print(news_df.target_names)
print(len(news_df.target_names)) # 8

print(news_df.target)
print(len(news_df.target)) # 7862


<class 'sklearn.utils.Bunch'>
dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

7862
I appreciate if anyone can point out some good books about the dead sea
scrolls of Qumran. Thanks in advance.
['/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/soc.religion.christian/20630'
 '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-test/sci.med/59422'
 '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-test/comp.graphics/38765'
 ...
 '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.baseball/102656'
 '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/sci.electronics/53606'
 '/home/jovyan/scikit_learn_data/20news_home/20news-bydate-train/talk.politics.mideast/76505']
['comp.graphics', 'comp.windows.x', 'rec.motorcycles', 'rec.sport.baseball', 'sci.electronics', 'sci.med', 'soc.religion.christian', 'talk.politics.mideast']
8
[6 5 0 ... 3 4 7]
7862


In [22]:
# import pandas as pd
# df = pd.DataFrame(news_df), index=news_df.data. columns=news_df.target_names) # ===> error keys들이 column으로 가기때문에
# df.head()

SyntaxError: invalid syntax (<ipython-input-22-b6fcb84863dd>, line 2)

In [6]:
feat_vect[0]

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [3]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [4]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

In [5]:
# 각 토피별로 연관도가 높은 순으로 word 나열
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)


Topic # 0
year 10 game medical health team 12 20 disease cancer 1993 games years patients good
Topic # 1
don just like know people said think time ve didn right going say ll way
Topic # 2
image file jpeg program gif images output format files color entry 00 use bit 03
Topic # 3
like know don think use does just good time book read information people used post
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war dos dos turkey arab armenia 000
Topic # 5
edu com available graphics ftp data pub motif mail widget software mit information version sun
Topic # 6
god people jesus church believe christ does christian say think christians bible faith sin life
Topic # 7
use dos thanks windows using window does display help like problem server need know run
